## Introduction
이 노트북은 Porto Serguro Competition에서 멋진 인사이트를 얻기 위해 작성됐습니다.   
뿐만 아니라 데이터 모델링을 준비하기 위한 tip과 trick들을 준비했습니다.   
목차는 다음과 같습니다:   
    1. Visual inspection of your data   
    2. Defining the meta data     
    3. Descriptive statistics     
    4. Handling imbalanced classes    
    5. Data quality checks    
    6. Exploratory data visualization    
    7. Feature engineering   
    8. Feature selection   
    9. Feature scaling   
    
## Loading packages

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

from sklearn.preprocessing import Impute가 from sklearn.impute import SimpleImputer로 바뀌었습니다.

## Loading data

In [46]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')

## Data at first sight
데이터에 대한 정보들:
   
   * 비슷한 Group에 태그된 Feature들은 비슷한 이름을 가지고 있습니다 (예를 들어, ind, reg, car, calc).
   * bin 이라는 접미사를 가진 Feature는 Binary feature임을 나타내고, cat 이라는 접미사를 가진 Feature는 Categorical feature임을 나타냅니다.
   * 이외의 Feature들은 Continious 혹은 Ordinal feature 입니다.
   * 값이 -1 인 관측치는 결측값(NaN)을 의미합니다
   * Target 열은 Policy holder에게 청구 적용 여부(Y/N)를 의미합니다  
    
중요한 정보에 대해 파악을 했습니다!
데이터의 전체적인 모습을 확인하기 위하여 앞부분(head)과 뒷부분(tail)을 먼저 확인해보도록 합니다.



In [47]:
train.head()

In [48]:
train.tail()

우리는 다음과 같은 정보를 확인할 수 있습니다.   

   * binary variables
   * 값이 정수로 이루어진 categorical variables
   * 값이 정수 혹은 소수로 이루어진 other variables
   * 관측값이 -1인 값은 결측치(NaN)를 나타냅니다.
   * Target Variables와 ID
   
shape를 활용하여 전체 데이터의 행과 열 개수를 확인합니다.


In [49]:
train.shape

59개의 variables와 595,212개의 row가 있습니다.
다음으로 test 데이터도 똑같은 수의 variables가 있는지 확인해봅니다.
그 전에, training 데이터에 중복값이 있는지 확인합니다.

In [50]:
train.drop_duplicates()
train.shape

중복값이 없습니다.

In [51]:
test.shape

training 데이터와 달리 test 데이터에는 58개의 variables가 있습니다.   
하지만 이 variable은 target variable이므로 상관 없습니다.   

이제는 우리가 가진 variables들의 타입을 조사해보도록 하겠습니다.   

추후에 우리는 14개의 Categorial variables를 더 미화시킬 것입니다.   
접미사로 bin이 붙은 variables는 이미 0과 1로 구성되어 있으므로 따로 더미화시킬 필요가 없습니다.   

In [52]:
train.info()

info() 메소드를 사용함으로써 우리는 다시 한 번 데이터들의 타입이 integer 혹은 float임을 확인했습니다. info() 메소드 상으로는 결측값이 없는 것으로 확인됩니다. 앞서 언급했듯이 결측값이 -1로 대치되어있기 때문입니다. 우리는 이를 나중에 다루도록 하겠습니다.

## Metadata
데이터 관리를 용이하게 하기 위해서, 우리는 variables의 정보들을 데이터프레임의 형태로 저장하고자 합니다. 이 메타 데이터들은 분석에 필요한 특정한 variables를 선택할 때, 시각화를 할 때, 모델링을 할 때 등에 도움이 될 것입니다.

구체적으로 우리가 저장해야 할 것들은 다음과 같습니다 :

 
* role: input, ID, target
* level: nominal, interval, ordinal, binary
* keep: True or False
* dtype: int, float, str

In [53]:
data = []
for f in train.columns:
    # Defining the role
    # target과 id를 지정해준 뒤, 나머지는 모두 input으로 지정합니다.
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
        
    # Definig the level
    # target과 bin은 binary, id와 cat은 nominal, 나머지는 데이터 타입에 따라 float과 int로 지정합니다.
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nomial'
    elif train[f].dtype == 'float64':
        level = 'interval'
    elif train[f].dtype == 'int64':
        level = 'ordinal'
        
    # id를 제외한 모든 variables를 True로 지정합니다.
    keep = True
    if f == 'id':
        keep = False
        
    # Defining the data type
    dtype = train[f].dtype
    
    # 모든 variable의 메타 데이터를 담은 딕셔너리를 생성합니다.
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

원본에서는 메타 데이터의 level을 구분할 떄 train[f].dtype == float과 같은 형식으로 코드가 작성됐습니다. 제 코드에서는 'float64'로 수정이 되어 있는데, 기존 코드대로 진행하면 dtype부분에서 원하는대로 데이터가 구분되지 않습니다. 데이터 형식 뒤에 64도 반드시 입력해줘야 합니다!

In [54]:
meta

예시로 level이 nominal인 데이터의 인덱스를 추출해봅니다.

In [55]:
meta[(meta.level == 'nomial') & (meta.keep)].index

role과 level에 따른 target의 수를 아래를 통해 확인해봅니다.

In [56]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

## Descriptive statistics
이제 데이터프레임에 describe 메소드를 사용하여 기술통계량을 살펴보도록 합니다. 그러나 desecribe 메소드는 categorical, id variable의 기술통계량은 계산해주지 않습니다. 따라서 추후에 catgorical variables를 살펴보도록 합니다.

메타 파일을 통하여 손쉽게 기술통계량을 계산할 수 있습니다.
### Interval variables

In [57]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

#### reg variables
   * 오직 ps_reg_03 만이 결측값을 가지고 있습니다.
   * variables 사이의 범위 (min to max)를 고려했을때, 우리는 추후에 스케일링(예를 들어 StandardScaler)을 적용할 수도 있습니다. 하지만 우리가 사용하고자 하는 분류기에 따라 다를 것입니다.

#### car variables
   * ps_car_12와 ps_car_14가 결측치를 가지고 있습니다.
   * 이 variables 역시, 스케일링이 필요해 보입니다.
   
#### calc variables
   * 결측치가 없습니다.
   * 이 variables는 최댓값이 0.9인 것으로 보아 일종의 비율인 것 같습니다.
   * 모든 3개의 calc variables는 매우 비슷한 분포를 가지고 있습니다.
   
전반적으로, interval variables들 간의 범위가 상대적으로 좁음을 확인 가능합니다. 아마도 데이터를 익명화시키기 위하여 몇몇 변환 작업(예를 들어 log)이 이미 적용된 것은 아닐까요?

기술통계를 살펴봄으로써 다음과 같은 결과를 얻었습니다.

   * Feature 내부에 결측치의 존재 유무
   * min과 max를 비교함으로써 스케일링의 필요성 판단
   * max값을 살펴봄으로써 변수의 값이 비율인지 판단
   
### Ordinal variables

In [58]:
v = meta[(meta.level == 'ordinal')&(meta.keep)].index
train[v].describe()

* 결측치를 가지는 variable은 ps_car_11입니다.
* 다른 범위를 가지는 값들에 대하여 스케일링이 필요해 보입니다.

### Binary variables

In [59]:
v = meta[(meta.level == 'binary')&(meta.keep)].index
train[v].describe()

* train 데이터의 target 평균은 0.0036448로 3.645%입니다. 결과값이 매우 불균형함을 알 수 있습니다.
* 평균값을 통해 대다수의 variables가 0이라고 결론내릴 수 있습니다.

## Handling imbalanced classes
위에 언급했듯이 target=1의 비율이 target=0 비율보다 매우 적습니다.(0.963 VS 0.036) 결과값이 불균형한 모델은 높은 정확도를 가지지만 실제로는 부가적인 value가 추가될 수 있습니다. 이러한 문제를 해결하기 위해서 두 가지 방법을 사용할 수 있습니다.

   * oversampling records with target=1
   * undersampling records with target=0
   
우리는 큰 training set을 가지고 있으므로, undersampling을 진행하겠습니다. 비율은 0.9:0.1로 지정합니다. 이를 통해 우리는 매우 불균형한 결과
값 데이터는 10% 미만임을 확인 가능하며, 10% 정도로 undersampling을 진행하는 것을 확인할 수 있습니다.

In [60]:
desired_apriori=0.10

# target value의 인덱스를 추출합니다.
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# target value의 기존 record 수를 구합니다.
nb_0 = len(train.loc[idx_0]) # 573518개
nb_1 = len(train.loc[idx_1]) # 21694개

# undersampling 비율을 계산하고 target == 0인 record수를 계산합니다.
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
# ((1-0.1)*573518) / (0.1*21694)
# undersampling_rate 계산 공식을 암기해둡시다.
undersampled_nb_0 = int(undersampling_rate*nb_0)

print('Rate to undersampling records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# shuffle을 활용하여 undersampling된 개수만큼의 samples를 가지는 nb=0을 무작위로 추출합니다.
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# 추출한 인덱스와 기존의 idx_1을 활용하여 리스트를 만듭니다.
idx_list = list(undersampled_idx) + list(idx_1)

# undersample된 데이터 프레임을 돌려받습니다.
train = train.loc[idx_list].reset_index(drop=True)

## Data Quality Checks
### Checking missing values
결측값은 -1로 나타내지고 있습니다.

In [61]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

* ps_car_03_vat과 ps_car_05_vat는 높은 결측치 비율을 가지고 있습니다. (68.39%, 44.26%) 따라서 삭제해주도록 합니다.
* 결측값이 있는 다른 cat variables는 결측값을 -1 그대로 둘 수 있습니다.
* ps_reg_03 (continuous)은 18%의 결측값을 지니고 있습니다. 평균값으로 대치해줍니다.
* ps_car_11 (ordinal)은 5개의 결측값을 지니고 있습니다. ordinal의 형태이므로 평균값으로 대치하면 안됩니다. 최빈값으로 대치해줍니다.
* ps_car_12 (continuous)은 1개의 결측값을 지니고 있습니다. 평균값으로 대치해줍니다.
* ps_car_14 (continuous)은 7%의 결측값을 지니고 있습니다. 평균값으로 대치해줍니다.

In [62]:
# 너무 많은 결측값을 지닌 Feature들을 제거합니다. (68.4%, 44.3%)
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop), 'keep'] = False # 메타데이터를 업그레이드해줍니다.

# 결측값을 Imputer를 활용하여 변환해줍니다.
mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

#### Checking the cardinality of the categorical variagles
범주성은 전체 행에 대한 특정 컬럼의 중복 수치를 나타내는 지표입니다. 중복도가 높으면 범주성이 낮으며, 중복도가 낮으면 범주성이 높습니다. 범주성은 상대적인 개념으로 이해해야 합니다.

따라서 범주성은 variable 내에서 다른 value의 개수를 말합니다. 우리는 추후 categorical variables를 더 미화시킬 것인데, variables 내에 다른 value들이 얼마나 많은지 체크해봐야 합니다. Value들이 많을 경우, 수많은 더미 변수들이 만들어질 수 있기 때문입니다.

In [63]:
v = meta[(meta.level == 'nomial')&(meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

합리적이긴 하지만, ps_car_11_cat는 104개로 매우 많은 Value를 가지고 있습니다.   
EDIT : 최초 작성자분은 104개의 Value에 대해 가공을 하여 데이터 손실이 있었던 것으로 보입니다. 이후 최초 작성자분은 Oliver의 커널을 활용한 방법을 사용했습니다.

In [64]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series*(1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                 tst_series=None,
                 target=None,
                 min_samples_leaf=1,
                 smoothing=1,
                 noise_level=0):
    '''
Smoothing is computed like in the following paper by Daniele Micci-Barreca
https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
trn_series : taining categorical feature as a pd.Series
tst_series : test categorical feature as a pd.Series
target : target data as a pd.Series
min_samples_leaf (int) : minimum samples to take category average into account
smoothin (int) : smoothing effect to balance categorical average vs prior
    '''
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count'])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages['mean'] * smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    # Apply average to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index' : target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

필자는 커널을 번역하면서 필사를 하는 중에 상기 과정이 정확히 어떻게 이루어지는지 이해하기 어렵습니다. 기본적으로 너무 Value가 많은 Categorical Variables를 가공하는 과정임은 여러분도 이해하고 계실 것입니다.   

따라서 함수의 과정을 하나하나 따라가보도록 하곘습니다.   
저처럼 이해가 안되셨던 분이라면 함께 하시면 좋을 것 같습니다.

In [65]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

Noise를 일으키는 함수를 정의한 것 같습니다.

시리즈값과 노이즈 레벨을 변수로 받아서, 시리즈 (1 + 노이즈레벨 표준정규분포로부터 샘플링된 난수)를 되돌려줍니다.

target encode 함수는 assert부터 진행하겠습니다.

In [66]:
assert len(train['ps_car_11_cat']) == len(train['target'])
assert train['ps_car_11_cat'].name == test['ps_car_11_cat'].name

저희가 사용할 trn_series는 train['ps_car_11_cat']이고, target은 train['target']입니다.   
두 시리즈의 길이가 같은지 확인하고, train['ps_car_11_cat']과 test['ps_car_11_cat']의 이름이 같은지도 확인해줍니다.

In [67]:
temp = pd.concat([train['ps_car_11_cat'], train['target']], axis=1)
print(temp)

두 시리즈를 열을 기준으로 concat 한 뒤, temp라는 이름의 변수로 저장해줍시다.

In [68]:
averages = temp.groupby(train['ps_car_11_cat'].name)[train['target'].name].agg(['mean', 'count'])
print(averages)

target열의 값들을 train['ps_car_11_cat'] 기준으로 그룹화한 뒤 mean함수와 count함수를 적용한 값을 averages 변수에 저장합니다.   
averages 함수는,   
각 value별 target 평균과 횟수 정보를 담고 있습니다.

In [69]:
smoothing = 1 / (1 + np.exp(-(averages['count'] - 100) / 10))
print(smoothing)

정확히 Smoothing 이 무슨 작업을 하는지는 좀 더 공부해봐야할 것 같습니다.

In [70]:
prior = train['target'].mean()

averages[train['target'].name] = prior * (1 - smoothing) + averages['mean'] * smoothing
print(averages)

prior값은 train['target']의 평균값으로 합니다. 앞서 저희는 undersampling을 통해 0과 1의 비율을 9:1로 맞추었기 때문에 prior값은 0.1이 됩니다.   
이후 averages에 target이라는 이름을 가진 열을 추가해줍니다. 값은 Smoothing을 활용하여 변환됩니다.

In [71]:
averages.drop(['mean', 'count'], axis=1, inplace=True)

필요한 값은 Smoothing한 값 뿐인 것 같습니다. drop을 활용하여 mean과 count를 빼줍니다.

In [72]:
ft_trn_series = pd.merge(
    train['ps_car_11_cat'].to_frame(train['ps_car_11_cat'].name),
    averages.reset_index().rename(columns={'index': train['target'].name, train['target'].name: 'average'}),
    on=train['ps_car_11_cat'].name,
    how='left')['average'].rename(train['ps_car_11_cat'].name + '_mean').fillna(prior)

print(ft_trn_series)

1. ps_car_11_cat 시리즈를 to_frame으로 데이터 프레임으로 가져옵니다.
2. averages의 인덱스를 초기화하고, 인덱스 값의 명칭을 target, target이었던 열이름을 average로 바꿔줍니다.
3. on값과 how값을 지정해주어 merge해줍니다.
4. 시리즈의 명칭을 ps_car_11_cat_mean으로 rename해주고 결측값은 prior값으로 대치해줍니다.

In [73]:
ft_trn_series.index = train['ps_car_11_cat'].index

새롭게 만든 시리즈의 인덱스를 기존 트레이닝 데이터 시리즈 인덱스와 맞춰줍니다.

In [74]:
ft_tst_series = pd.merge(test['ps_car_11_cat'].to_frame(test['ps_car_11_cat'].name),
                        averages.reset_index().rename(columns={'index' : 'target', 'target' : 'averages'}),
                        on=test['ps_car_11_cat'].name,
                        how='left')['averages'].rename(train['ps_car_11_cat'].name + '_mean').fillna(prior)

ft_tst_series.index = test['ps_car_11_cat'].index

앞서 train 데이터에서 진행했던 작업을 test 데이터에서도 그대로 진행해줍니다.

In [75]:
add_noise(ft_trn_series, 0.01), add_noise(ft_tst_series, 0.01)

처음에 정의했던 add_noise 함수를 활용하여 노이즈를 일으킨 값들을 반환받습니다. 함수에서 진행한 과정들을 요약해보면 다음과 같습니다.   
1. noise를 만들어줄 add_noise 함수 정의
2. train 데이터와 target 데이터의 len이 같은지, test 데이터와 train 데이터의 이름이 같은지 확인
3. train 시리즈와 target 시리즈를 concat
4. value별 mean과 count 계산하여 Averages로 저장
5. Smoothing을 계산
6. prior를 target 데이터의 평균값으로 정의
7. 앞서 진행했던 Value별 평균에 Smoothing을 진행하고 필요 없어진 mean과 count 제거
8. Averages의 값으로 새로운 시리즈(trn/tst_cat_mean)정의
9. 최초 정의한 add_noise를 적용한 시리즈 반환

조금 더 필사해보면서 이해하도록 합시다.

In [76]:
train_encoded, test_encoded = target_encode(train['ps_car_11_cat'],
                                           test['ps_car_11_cat'],
                                           target=train.target,
                                           min_samples_leaf=100,
                                           smoothing=10,
                                           noise_level=0.01)
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat', 'keep'] = False # Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

## Exploratory Data Visualization

### Categorical variables
target값이 1인 categorical variables와 customers의 비율을 살펴보도록 합시다.

In [77]:
v = meta[(meta.level == 'nomial')&(meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20,10))
    # Calculate the percentage of target=1 per category value
    cat_perc = train[[f, 'target']].groupby([f], as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.show()

막대 그래프들을 통해 결측값이 있는 variable들을 확인할 수 있습니다. 앞서 결측값들을 치환했는데, categorical variable들은 따로 치환을 하지 않았습니다. 최빈값으로 대체하는 것보다 분리된 category value로서 결측값을 보는 것이 더 좋은 방법일 수 있습니다.

결측값을 가지고 있는 Customer들이 다른 Value들에 비하여 훨씬 높은 target 평균을 가지고 있기 때문입니다!

### Interval variables
interval variables의 상관관계를 확인하고자 합니다. Heatmap은 variables 간의 상관관계를 확인하는데 매우 효율적입니다. 코드는 an example by Michael Waskon=m에 기반하고 있습니다.

In [78]:
def corr_heatmap(v):
    correlations = train[v].corr()
    
    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
               square=True, linewidths=.5, annot=True, cbar_kws={'shrink': .75})
    plt.show()
    
v = meta[(meta.level == 'interval')&(meta.keep)].index
corr_heatmap(v)

강한 상관관계를 가지고 있는 variable들은 다음과 같습니다.:

* ps_reg_01 & ps_reg_03 (0.7)
* ps_car_12 & ps_car_13 (0.67)
* ps_car_12 & ps_car_14 (0.58)
* ps_car_13 & ps_car_15 (0.67)

Seaborn의 pairplot을 사용하면 variable들의 (선형)관계를 손쉽게 시각화할 수 있습니다. 하지만 히트맵이 상관관계가 있는 variable들의 관계들을 시각화해주고 있기 때문에, 우리는 높은 상관관계를 보이는 variable들을 분리해서 보고자 합니다.

Note 프로세스의 속도를 높이기 위하여 train 데이터의 sample을 사용합니다.

In [79]:
s = train.sample(frac=0.1)

train 데이터에서 10%의 데이터를 샘플링합니다.

* train.shape -> (216940, 57)
* s.shape -> (21694, 57)

#### ps_reg_02 & ps_reg_03 (0.7)
회귀선이 보여주듯이, 두 variable들 간에는 선형 상관관계를 살펴볼 수 있습니다. hue 파라미터를 통해 target=0과 target=1에 대한 회귀선이 동일함을 알 수 있습니다.

In [80]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

#### ps_car_12 & ps_car_13 (0.67)

In [81]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

#### ps_car_12 & ps_car_14 (0.58)

In [82]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

#### ps_car_13 & ps_car_15 (0.67)

In [83]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

이제는 어떤 correlated variables를 유지할지 결정해야 합니다. 이를 위하여 우리는 Principal Component Analysis(PCA), 주성분 분석을 실시하여 variables의 dimensions를 줄일 수 있습니다. 하지만 correlated variables의 수가 적은만큼, 우리는 모델이 heavy-lifting을 하도록 해야합니다.

#### Checking the correlations between ordinal variables

In [84]:
v = meta[(meta.level == 'ordinal')&(meta.keep)].index
corr_heatmap(v)

ordinal variables는 큰 상관관계를 가지고 있지 않은 것으로 보입니다. 반면에 target 값으로 그룹화할 때 분포가 어떻게 될지 확인할 수 있습니다.

### Feature engineering

#### Creating dummy variables
categorical variables는 어떤 순서나 경중이 담겨있지 않습니다. 예를 들어서 카테고리 2는 카테고리 1보다 2배의 값을 가지고 있지 않습니다. 이 문제는 더미 데이터를 만들어줌으로써 해결할 수 있습니다. 첫번째 dummy variables의 정보는 원래 variables의 범주에 대해 생성된 다른 dummy variable에서 파생될 수 있으므로 삭제해주도록 합니다.

In [85]:
v = meta[(meta.level == 'nominal')&(meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

dummy variables는 training 데이터 세트에 52개의 variables를 추가했습니다.

#### Creating interavtion variables

In [86]:
v = meta[(meta.level == 'interval')&(meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names(v))
interactions.drop(v, axis=1, inplace=True)
# poly 처리가 되지 않은 기존의 열들을 삭제합니다.

# interactions와 train 데이터를 합쳐줍니다.
print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train = pd.concat([train, interactions], axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

PolynomialFeatures는 다항차수 변환 진행을 도와주는 클래스입니다. 상기 코드의 경우 degree를 2로 설정했으니 2차항 변수로 만들어주는 것입니다.

이를 통해 train 데이터에 interation variables를 추가할 수 있습니다.
get_feature_names 메소드 덕분에 열 이름을 할당할 수 있습니다.

### Feature selection

#### Removing features with low or zero variance

개인적으로 작성자는 분류기의 알고리즘이 유지할 features를 선택하는 것을 선호한다고 합니다. 하지만 우리 스스로 할 수 있는 일도 있습니다. 분산이 0이거나 아주 적은 feature들을 제거하는 것입니다.

이를 위해 사이킷런의 VarianceThreshold라는 메소드를 사용할 수 있습니다. 기본적으로 이 메소드는 분산 값이 0인 feature들을 제거해줍니다.

하지만 저희는 이전 단계에서 이미 분산이 0인 feature가 없음을 확인했기 때문에, 우리는 1% 미만의 분산이 있는 feature들을 제거해주고자 합니다. 이를 통해 우리는 31개의 variance를 제거하게 됩니다.

In [87]:
selector = VarianceThreshold(threshold=.01)
selector.fit(train.drop(['id', 'target'], axis=1))
# fit to train without id and target variables

f = np.vectorize(lambda x : not x)
# function to toggle boolean array elements

v = train.drop(['id', 'target'], axis=1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

만약 우리가 분산에 기반하여 선택을 진행한다면 많은 variable들을 잃게 될 것입니다. 하지만 우리는 많은 variables를 가지고 있지 않기 때문에, 분류기가 직접 선택하도록 합니다. variables가 더 많은 데이터셋이라면 처리 시간을 줄여줄 수 있을 것입니다.

사이킷런은 [feature selection methods]를 제공합니다. 이 메소드 중 하나가 'SelectModel'인데, 다른 분류기에서 최상의 feature를 선택하고 기능을 계속할 수 있도록 합니다. 아래를 통해 랜덤 포레스트를 어떻게 사용하는지 확인해보도록 합니다.

#### Selecting features with a Random Forest and SelectFromModel
우리는 랜덤 포레스트의 feature importances에 따라 feature 선택의 기준을 삼습니다. SelectFromModel을 통하여 유지할 variables의 숫자를 구체화할 수 있습니다. feature의 중요도에 대한 임곗값을 수동으로 설정할 수 있지만, 우리는 단순히 50% 이상의 최적의 variables를 선택해보도록 합시다.

하기의 코드는 이곳에서 가져왔습니다.  GitHub repo of Sebastian Raschka.

In [88]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print('%2d) %-*s %f' % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

In [89]:
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection : {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [90]:
train = train[selected_vars + ['target']]

### Feature scaling
이전에 언급했듯이, 우리는 train 데이터에 정규화를 진행할 수 있습니다. 몇몇 분류기에서는 더 나은 결과를 가져올 수 있을 것입니다.

In [91]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis=1))